In [1]:
import pandas as pd
from prediction.model import TrajectoryTransformer, TrajectoryTrainer
from prediction.data import MaskedAISDataset

from torch.utils.data import DataLoader

/Users/marcus/Documents/Workspace/python-envs/ais-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_pickle('ais_data.pkl')

In [3]:
MAX_SEQ_LEN = 100
dataset = MaskedAISDataset(df, MAX_SEQ_LEN, n_workers=1, normalize=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)
device = "mps"

transformer = TrajectoryTransformer(
    d_model=128,
    nhead=4,
    num_encoder_layers=4,
    num_decoder_layers=4,
    dim_feedforward=512,
    dropout=0.1,
    max_seq_len=MAX_SEQ_LEN
)

trainer = TrajectoryTrainer(transformer, loader, device=device, use_wandb=False)

100%|██████████| 8585/8585 [00:04<00:00, 2003.18it/s]
/Users/marcus/Documents/Workspace/python-envs/ais-env/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [4]:
trainer.train(num_epochs=10)


Epoch 1/10


Training: 100%|██████████| 269/269 [00:20<00:00, 13.05it/s, loss=0.0903]



Epoch 2/10


Training: 100%|██████████| 269/269 [00:16<00:00, 16.78it/s, loss=0.0742]



Epoch 3/10


...

Training: 100%|██████████| 269/269 [00:16<00:00, 16.56it/s, loss=0.0326]



Epoch 9/10


Training: 100%|██████████| 269/269 [00:16<00:00, 16.58it/s, loss=0.0442]



Epoch 10/10


Training: 100%|██████████| 269/269 [00:16<00:00, 16.70it/s, loss=0.0665]
